In [2]:
import numpy as np
import os
from pprint import pprint

from collections import defaultdict

1. Превратить монофонное выравнивание в трифонное выравнивание.
2. Оценить параметры гауссиан для каждой из трифон.
3. Кластеризовать похожие трифоны с общей серединой. Можно использовать sklearn.cluster. Попробуйте подобрать наиболее оптимальное количество классов.

4*. Для каждого кластера трифон (кстати, называются они сенонами) натренируйте параметры для смеси гауссиан при помощи EM-алгоритма.
1. От монофон стоит отрезать суффиксы _B, _I, _E, _S там, где они есть. Это облегчит конвертацию монофонных выравниваний в трифонные.
2. Для каждого трифона оцените mean, std, сконкатенируйте в единый 80-мерный вектор, и уже эти вектора кластеризуйте!
3. Кластеризовать нужно группы трифонов с общим средним монофоном. Чтобы много не думать, будем считать, что каждую группу нужно кластеризовать на 5 классов.
4. Посмотрите на полученные кластера и постарайтесь понять, почему они такими получились!

In [3]:
filenames = os.listdir('np_data')
print(len(filenames))

5406


In [4]:
uids = list(set([fname.split('.')[0] for fname in filenames]))
print(uids[:5])

['8297-275154-0008', '8297-275154-0009', '2277-149896-0018', '2277-149896-0019', '2277-149896-0014']


In [5]:
feats = {}
alis = {}
for uid in uids:
  feats[uid] = np.load('np_data/{}.feats.npy'.format(uid))
  alis[uid] = np.load('np_data/{}.targs.npy'.format(uid))

Создадим словарь соответствия фонем и их идентификаторов.

In [6]:
phones = {}
f = open("data/lang/phones.txt")
for line in f:
    line = line.strip().split()
    ph = line[0].split("_")[0]
    phones[int(line[1])] = ph
f.close()

In [7]:
# Проверим:
key = '2277-149896-0017'
a = [phones[a] for a in alis[key]]
print a

['SIL', 'SIL', 'SIL', 'SIL', 'SIL', 'SIL', 'SIL', 'SIL', 'SIL', 'SIL', 'SIL', 'SIL', 'SIL', 'SIL', 'SIL', 'SIL', 'SIL', 'SIL', 'SIL', 'SIL', 'SIL', 'SIL', 'SIL', 'SIL', 'SIL', 'SIL', 'SIL', 'SIL', 'SIL', 'SIL', 'SIL', 'SIL', 'SIL', 'SIL', 'SIL', 'SIL', 'SIL', 'SIL', 'SIL', 'SIL', 'SIL', 'SIL', 'SIL', 'SIL', 'SIL', 'SIL', 'SIL', 'SIL', 'SIL', 'SIL', 'SIL', 'AE1', 'AE1', 'AE1', 'AE1', 'AE1', 'AE1', 'AE1', 'AE1', 'AE1', 'AE1', 'T', 'T', 'T', 'T', 'T', 'T', 'W', 'W', 'W', 'W', 'W', 'W', 'AH1', 'AH1', 'AH1', 'AH1', 'AH1', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'TH', 'TH', 'TH', 'TH', 'TH', 'TH', 'EU1', 'EU1', 'EU1', 'EU1', 'EU1', 'EU1', 'R', 'R', 'R', 'R', 'D', 'D', 'D', 'D', 'IY0', 'IY0', 'IY0', 'IY0', 'IY0', 'IY0', 'IY0', 'IY0', 'IY0', 'HH', 'HH', 'HH', 'HH', 'IY1', 'IY1', 'IY1', 'IY1', 'IY1', 'IY1', 'W', 'W', 'W', 'W', 'W', 'W', 'W', 'EH1', 'EH1', 'EH1', 'EH1', 'EH1', 'EH1', 'N', 'N', 'N', 'N', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'AH0', 'AH0', 'AH0', 'AH0', 'AH0', 'AH0', '

Напишем функцию для извлечения трифонов и их векторов.

In [8]:
i = a.index("SIL")
SIL_vector = feats[key][i]

def build_triphones(ali, feat, phones):
    ali = [phones[l] for l in ali]
    triphones_vector = []
    for i in range(0, len(ali)):
        triphone = []
        vector = []
        
        j = 1
        while True:
            if i - j < 0:
                triphone.append("SIL")
                vector.append(SIL_vector)
                break
            left_context = ali[i - j]
            
            if left_context == ali[i]:
                j += 1
            else:
                triphone.append(left_context)
                vector.append(feat[i - j])
                break

        triphone.append(ali[i])
        vector.append(feat[i])
        j = 1
        while True:
            try:
                right_context = ali[i + j]
            except IndexError:
                triphone.append("SIL")
                vector.append(SIL_vector)
                break

            if right_context == ali[i]:
                j += 1
            else:
                triphone.append(right_context)
                vector.append(feat[i + j])
                break
        
        triphones_vector.append(("_".join(triphone), vector))
    return triphones_vector

Построим распределение p2f

In [9]:
all_triphones = []
p2f = defaultdict(list)
for uid in uids:
    triphones = build_triphones(alis[uid], feats[uid], phones)
    for triphone in triphones:
        all_triphones.append(triphone[0])
        for t in triphone[1]:
            p2f[triphone[0]] += [t]

In [11]:
def mu_sigma(feats):
  mean = np.mean(feats, axis=0)
  std = np.std(feats, axis=0)
  return mean, std

In [12]:
means = {}
stds = {}
for phone in p2f:
  means[phone], stds[phone] = mu_sigma(p2f[phone])

Посмотрим, сколько у нас всего трифонов.

In [93]:
print len(p2f)

17380


Теперь соединим векторы means и stds.

In [94]:
vectors = {}
for key in means:
    vectors[key] = np.hstack((means[key], stds[key]))

Разделим трифоны на группы трифонов с общей серединой.

In [88]:
middle = []
for key in vectors:
    middle.append(key.split("_")[1])

middle = list(set(middle))
print middle

['UW0', 'AO1', 'DH', 'IH0', 'IH1', 'AH0', 'B', 'JH', 'UH0', 'UH1', 'OY0', 'SIL', 'EH0', 'EH1', 'NG', 'EY1', 'EY0', 'TH', 'AA1', 'AY1', 'EU0', 'AH1', 'CH', 'ZH', 'D', 'G', 'F', 'K', 'M', 'L', 'AW1', 'N', 'P', 'S', 'R', 'T', 'W', 'V', 'Y', 'Z', 'OW1', 'OW0', 'AO0', 'AE1', 'AE0', 'UW1', 'EU1', 'AW0', 'HH', 'OY1', 'AY0', 'SH', 'IY1', 'IY0']


In [89]:
classes = defaultdict(list)
for m in middle:
    classes[m] = [key for key in vectors if key.split("_")[1] == m]
print len(classes)

train_set = defaultdict(tuple)
for cl in classes:
    features = []
    labels = []
    for triphone in classes[cl]:
        features.append(vectors[triphone])
        labels.append(triphone)
    train_set[cl] = (labels, features)
len(train_set)

54


54

Получился датасет, разбитый на 54 группы. К каждой группе принадлежат трифоны с одинаковой фонемой в середине.

Вот, например, все трифоны, имеющие середину "OW0":

In [90]:
print train_set['OW0'][0]

['G_OW0_IH1', 'L_OW0_SIL', 'R_OW0_AE1', 'D_OW0_S', 'D_OW0_Y', 'D_OW0_Z', 'AY1_OW0_B', 'AY1_OW0_M', 'S_OW0_IH1', 'S_OW0_AO1', 'L_OW0_AE1', 'R_OW0_DH', 'IY1_OW0_G', 'SIL_OW0_F', 'D_OW0_IY0', 'IY0_OW0_B', 'IY0_OW0_F', 'IY0_OW0_S', 'IY0_OW0_T', 'HH_OW0_SIL', 'M_OW0_Z', 'S_OW0_HH', 'P_OW0_P', 'UW1_OW0_B', 'K_OW0_Z', 'T_OW0_M', 'T_OW0_S', 'T_OW0_Z', 'V_OW0_K', 'L_OW0_Y', 'L_OW0_Z', 'HH_OW0_T', 'K_OW0_HH', 'Z_OW0_G', 'P_OW0_EH1', 'R_OW0_AH1', 'D_OW0_SIL', 'N_OW0_SIL', 'R_OW0_F', 'D_OW0_EH1', 'R_OW0_HH', 'S_OW0_IH0', 'R_OW0_IH0', 'Z_OW0_AA1', 'D_OW0_HH', 'L_OW0_B', 'L_OW0_D', 'L_OW0_W', 'K_OW0_R', 'K_OW0_S', 'K_OW0_P', 'K_OW0_W', 'K_OW0_G', 'K_OW0_D', 'K_OW0_K', 'L_OW0_AH0', 'L_OW0_AW1', 'N_OW0_T', 'N_OW0_Z', 'N_OW0_F', 'K_OW0_SIL', 'G_OW0_HH', 'R_OW0_SIL', 'N_OW0_V', 'M_OW0_B', 'G_OW0_P', 'N_OW0_B', 'N_OW0_K', 'L_OW0_EU0', 'S_OW0_AH1', 'L_OW0_DH', 'S_OW0_DH', 'D_OW0_P', 'D_OW0_B', 'D_OW0_L', 'K_OW0_IH0', 'N_OW0_SH', 'Z_OW0_HH', 'D_OW0_AH1', 'R_OW0_P', 'R_OW0_Z', 'S_OW0_B', 'S_OW0_T', 'R_OW0_A

Теперь кластеризуем каждую группу.

In [91]:
from sklearn.cluster import KMeans

clusters = defaultdict(list)

for group in train_set:
    if len(train_set[group][1]) <= 5:
        continue
    kmeans = KMeans(n_clusters=5, random_state=0).fit(train_set[group][1])
    clusters[group] = kmeans.labels_
    
    print "Class: " + group
    print "Labels:"
    print kmeans.labels_ 
    for i in range(0,5):
        print
        print "Cluster " + str(i + 1) + ":"
        labels = train_set[group][0]
        print [labels[j] for j in range(0, len(labels)) if kmeans.labels_[j] == i]
        print
    
    print '======================================'

Class: AO1
Labels:
[0 3 3 4 3 3 1 3 1 1 0 4 1 2 0 0 1 0 4 2 1 1 0 2 4 0 3 0 2 0 0 4 2 4 3 0 0
 1 3 2 1 1 2 0 0 3 4 2 3 0 1 2 3 2 0 0 4 0 1 4 3 4 0 0 2 3 0 0 0 4 2 0 0 1
 3 0 2 2 4 4 3 0 3 0 4 1 0 4 1 1 0 2 0 1 2 1 3 3 0 2 2 3 1 1 4 1 1 4 2 0 0
 0 2 4 4 2 2 3 3 3 3 4 3 3 3 2 3 0 1 3 3 4 0 4 1 3 2 4 0 0 2 1 4 2 0 3 1 1
 2 0 2 1 1 4 0 1 2 4 1 2 0 0 0 0 4 0 4 0 2 1 1 1 0 0 1 0 4 0 1 0 4 3 3 0]

Cluster 1:
['W_AO1_K', 'IY0_AO1_T', 'IY0_AO1_K', 'IY0_AO1_F', 'NG_AO1_F', 'G_AO1_F', 'UW1_AO1_T', 'B_AO1_CH', 'M_AO1_F', 'M_AO1_S', 'SIL_AO1_T', 'UW1_AO1_F', 'AY1_AO1_F', 'AY1_AO1_T', 'IY0_AO1_TH', 'P_AO1_F', 'P_AO1_Z', 'P_AO1_S', 'Z_AO1_S', 'Z_AO1_F', 'K_AO1_Z', 'K_AO1_T', 'K_AO1_S', 'K_AO1_K', 'K_AO1_F', 'D_AO1_TH', 'D_AO1_F', 'L_AO1_SH', 'OW0_AO1_F', 'EY1_AO1_T', 'K_AO1_SH', 'OW1_AO1_F', 'SIL_AO1_N', 'SIL_AO1_G', 'SIL_AO1_S', 'SIL_AO1_F', 'L_AO1_S', 'B_AO1_S', 'D_AO1_Z', 'L_AO1_F', 'AW1_AO1_F', 'IY1_AO1_F', 'T_AO1_K', 'T_AO1_F', 'T_AO1_T', 'T_AO1_S', 'R_AO1_T', 'R_AO1_S', 'R_AO1_F', 'R_AO1_G', 'R

Чтобы посмотреть, почему кластеры разбились именно так, в каждом классе я буду выделять наиболее частотные начала и окочания трифона (первая и третья фонемы). Чтобы сделать нормализацию, для начала нужно посчитать частотность каждой фонемы (т.е. как часто фонема встречается в начале трифона и как часто фонема встречается в конце трифона).

In [57]:
from collections import Counter

# ct = Counter(all_triphones)
beginnings = [tr.split("_")[0] for tr in all_triphones]
ends = [tr.split("_")[2] for tr in all_triphones]
beginnings = Counter(beginnings)
ends = Counter(ends)


Выведем наиболее часто встречающиеся в начале трифона фонемы:

In [58]:
for key in sorted(beginnings, key=lambda x: beginnings[x], reverse=True)[:20]:
    print(key, beginnings[key])

('SIL', 196012)
('R', 127368)
('N', 126787)
('AH0', 124760)
('T', 114310)
('D', 91385)
('S', 84116)
('L', 81294)
('IH0', 63602)
('Z', 59516)
('M', 56921)
('AH1', 45021)
('EH1', 44781)
('IY0', 44549)
('IH1', 41989)
('K', 40563)
('DH', 39526)
('AE1', 34903)
('IY1', 34380)
('HH', 32324)


И теперь выведем фонемы, наиболее часто встречающиеся в конце трифона:

In [59]:
for key in sorted(ends, key=lambda x: ends[x], reverse=True)[:20]:
    print(key, ends[key])

('SIL', 226111)
('N', 115646)
('T', 106071)
('R', 105247)
('AH0', 97816)
('S', 74902)
('AE1', 69904)
('D', 69231)
('DH', 68944)
('L', 66775)
('IH0', 59533)
('EH1', 51606)
('M', 47407)
('AH1', 47197)
('Z', 46831)
('HH', 46181)
('W', 44235)
('IH1', 43378)
('K', 43116)
('AY1', 38649)


Теперь для каждого класса трифон (трифоны с одинаковой серединой), посмотрим, как внутри каждого кластера распределились фонемы в начале и конце трифонов:

In [84]:
print clusters.keys()

['W', 'DH', 'IH0', 'UW0', 'AH0', 'AH1', 'JH', 'UH0', 'UH1', 'SIL', 'EH0', 'EH1', 'NG', 'EY1', 'EY0', 'TH', 'IY1', 'AY1', 'EU0', 'CH', 'ZH', 'D', 'G', 'F', 'K', 'M', 'L', 'AW1', 'N', 'P', 'S', 'R', 'T', 'AO1', 'V', 'Y', 'Z', 'OW1', 'OW0', 'AO0', 'AE1', 'AE0', 'UW1', 'EU1', 'HH', 'OY1', 'AY0', 'IH1', 'SH', 'AA1', 'IY0', 'B']


In [92]:
samples = list(clusters.keys())
print(samples)

for sample in samples:
    print "Triphone group: ", sample
    partition = clusters[sample]
    
    print "Group size: ", len(partition)
    groups = []
    for i in range(0, 5):
        labels = train_set[sample][0]
        groups.append([labels[j] for j in range(0, len(labels)) if partition[j] == i])

    for i in range(0, 5):
        print"Cluster: " + str(i + 1)
        bs = []
        es = []
        for triphone in groups[i]:
            b, _, e = triphone.split("_")
            bs.append(b)
            es.append(e)
        bs = Counter(bs)
        es = Counter(bs)
        for key in bs:
            bs[key] = float(bs[key])/float(beginnings[key])
        for key in es:
            es[key] = float(es[key])/float(ends[key])
        
        print("Most popular beginnings of triphones")
        for key in sorted(bs, key=lambda x: bs[x], reverse=True)[:7]:
            print(key, bs[key])
        print('....')
        print("Most popular ends of triphones")
        for key in sorted(es, key=lambda x:es[x], reverse=True)[:7]:
            print(key, es[key])
        print("------------")
        print
    print
    print('=================================================')

['W', 'DH', 'IH0', 'UW0', 'AH0', 'AH1', 'JH', 'UH1', 'SIL', 'EH0', 'EH1', 'NG', 'EY1', 'EY0', 'TH', 'IY1', 'AY1', 'EU0', 'CH', 'ZH', 'D', 'G', 'F', 'K', 'M', 'L', 'AW1', 'N', 'P', 'S', 'R', 'T', 'AO1', 'V', 'Y', 'Z', 'OW1', 'OW0', 'AO0', 'AE1', 'AE0', 'UW1', 'EU1', 'HH', 'OY1', 'AY0', 'IH1', 'SH', 'AA1', 'IY0', 'B']
Triphone group:  W
Group size:  351
Cluster: 1
Most popular beginnings of triphones
('ZH', 0.0018115942028985507)
('CH', 0.000977092606665943)
('JH', 0.0007376444553725104)
('SH', 0.00037355248412401944)
('Z', 0.00018482424894146113)
('TH', 0.00017161489617298782)
('S', 0.00015454848066955156)
....
Most popular ends of triphones
('ZH', 0.001272264631043257)
('CH', 0.0010504201680672268)
('JH', 0.0007817589576547231)
('SH', 0.000259773996622938)
('Z', 0.0002348871474023617)
('TH', 0.00019364833462432224)
('S', 0.00017356011855491175)
------------

Cluster: 2
Most popular beginnings of triphones
('OW0', 0.000584966364434045)
('UW1', 0.00024652401143871415)
('NG', 0.0002326447

## Интерпретация

Видно, что в целом кластеры группируются вокруг фонем, схожих по месту образования (с т.з. фонетики).

1. Кластеры разделяются по характеру фонемы в начале и в конце. Видно, что чаще согласные группируются с согласными, а гласные – с гласными. В переходной зоне между согласными и гласными находятся сонорные согласные – M, L, R, NG, а также Y.

2. В один кластер часто попадают гласные одного ряда.

    В следующем примере сгруппированы вместе гласные заднего ряда и наиболее близкие к ним гласные среднего ряда (AY0)


        ('UW0', 0.0019404915912031048)

        ('AY0', 0.001893939393939394)

        ('AW1', 0.0006522549385016772)

        ('OY1', 0.00047214353163361664)

        ('UW1', 0.00044374322058968543)

        ('OW1', 0.0003322259136212625)

        ('OW0', 0.0002924831822170225)


    В другом кластере этой же группы сгрупированы сонорные согласные.

        ('M', 0.00010540925141863284)

        ('L', 9.840824661106601e-05)

        ('OW1', 9.492168960607498e-05)

        ('NG', 9.305788200260562e-05)

        ('IY0', 8.978877191407215e-05)

        ('R', 8.63639218642045e-05)

        ('AY1', 6.403278478581034e-05)
    
    3. В один кластер часто попадают трифоны с шумными согласными.

        ('ZH', 0.005434782608695652)

        ('CH', 0.0011942242970361524)

        ('JH', 0.0008605851979345956)

        ('TH', 0.0007722670327784452)

        ('SH', 0.0004669406051550243)

        ('DH', 0.0004553964479077063)

        ('K', 0.00029583610679683456)
    4. Первая и третья фонемы трифонов в одном кластере чаще всего очень похожи. Т.е. если первая фонема трифона – гласный звук, то и третья фонема, скорее всего, будет гласной. И наоборот. Не совсем понятно, как это интерпретировать, но, возможно, это происходит из-за того, что для английского языка нехарактерно скопление звуков одного типа, согласных или гласных, обычно согласные и гласные звуки чередуются.